In [ ]:
!nvidia-smi

Sun Jun 20 04:53:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip3 install transformers

     |████████████████████████████████| 2.5MB 8.4MB/s 
     |████████████████████████████████| 901kB 47.5MB/s 
     |████████████████████████████████| 3.3MB 44.5MB/s 


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch 
from transformers import BertModel,BertConfig,AutoModel,AutoTokenizer,AdamW,get_linear_schedule_with_warmup,get_constant_schedule
import pandas as pd 
import numpy as np 
from torch.utils.data import Dataset,DataLoader
from torch import nn,optim 
import seaborn as sns 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
# from fairseq.models.roberta import RobertaModel
# from fairseq.data.encoders.fastbpe import fastBPE
import random
import os 
from matplotlib import pyplot as plt 
from torch.nn.utils.rnn import pad_sequence
import pickle
from sklearn.metrics import accuracy_score

In [ ]:
!cp /content/drive/MyDrive/Viettel3/train_n.csv /content
!cp /content/drive/MyDrive/Viettel3/test_n.csv /content

In [ ]:
train=pd.read_csv('train_n.csv')
test=pd.read_csv('test_n.csv')

In [ ]:
n_samples=len(train)
weights[]
for i in range(10):
    weights.append(len(train[train.label==i]))
weights=[np.log10(n_samples/value) for value in weights]
weight=torch.tensor(weights)

In [ ]:
phobert=AutoModel.from_pretrained("vinai/phobert-base")
tokenizer=AutoTokenizer.from_pretrained("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
val=pd.DataFrame()
n_classes=len(set(train.label))
for i in range(n_classes):
    temp=train[train.label==i]
    temp=temp.sample(int(len(temp)*0.2))
    val=pd.concat([val,temp],axis=0)

train=train[~train.index.isin(val.index)]
train.to_csv("train.csv",index=False)
val.to_csv("val.csv",index=False)

In [ ]:
!cp /content/train.csv /content/drive/MyDrive/Viettel3/Bert
!cp /content/val.csv /content/drive/MyDrive/Viettel3/Bert

In [ ]:
train=pd.read_csv('train.csv')
val=pd.read_csv('val.csv')

In [ ]:
labels={
0:"Van hoa",
1:"Chinh tri Xa hoi",
2:"Khoa hoc",
3:"Phap luat",
4:"Vi tinh",
5:"Suc khoe",
6:"Kinh doanh",
7:"The thao",
8:"The gioi",
9:"Doi song"
}

In [ ]:
max_sequence_length=256

def encode_data(data):
  result=[]
  for text in data.text.values:
    temp=tokenizer.encode(text)
    if len(temp)<max_sequence_length:
      temp+=[1]*(max_sequence_length-len(temp))
    else:
      temp=temp[:max_sequence_length]
      temp[-1]=tokenizer.eos_token_id
    result.append(temp)
  return result

train['data']=encode_data(train)
test['data']=encode_data(test)
val['data']=encode_data(val)

Token indices sequence length is longer than the specified maximum sequence length for this model (325 > 256). Running this sequence through the model will result in indexing errors


In [ ]:
train_dataset_tensor=torch.utils.data.TensorDataset(torch.tensor(train['data'],dtype=torch.long),torch.tensor(train['label'],dtype=torch.long))
test_dataset_tensor=torch.utils.data.TensorDataset(torch.tensor(test['data'],dtype=torch.long),torch.tensor(test['label'],dtype=torch.long))
val_dataset_tensor=torch.utils.data.TensorDataset(torch.tensor(val['data'],dtype=torch.long),torch.tensor(val['label'],dtype=torch.long))

In [ ]:
train_loader=DataLoader(train_dataset_tensor,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset_tensor,batch_size=32,shuffle=True)
val_loader=DataLoader(val_dataset_tensor,batch_size=32,shuffle=True)

In [ ]:
print(len(train_loader))
print(len(test_loader))
print(len(val_loader))

845
1575
211


In [ ]:
class Model(nn.Module):
  def __init__(self,n_classes,drop_prob=0.2):
    super(Model,self).__init__()
    self.n_classes=n_classes
    self.bert=phobert
    self.lstm=nn.LSTM(input_size=self.bert.config.hidden_size,
                      hidden_size=512,
                      batch_first=True,
                      num_layers=2,
                      bidirectional=True)
    self.linear=nn.Sequential(
        nn.Linear(512*4,512),
        nn.ReLU(),
        nn.Dropout(drop_prob),
        nn.Linear(512,self.n_classes)
    )
      
  def forward(self,x_batch):
    word_embedding = self.bert(x_batch)[0]
    lstm_out,_=self.lstm(word_embedding)
    avg_hidden=torch.mean(lstm_out,dim=1)
    max_hidden,_=torch.max(lstm_out,dim=1)
    input_linear=torch.cat((avg_hidden,max_hidden),dim=1)
    out=self.linear(input_linear)
    return out


In [ ]:
n_epochs=100
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes=len(set(train.label))
model=Model(num_classes).to(device)
optimizer=AdamW(model.parameters(),lr=1e-5)
scheduler=get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=100,
    num_training_steps=len(train_loader)*(n_epochs-1)
)
scheduler_frozen=get_constant_schedule(optimizer)
criterion=nn.CrossEntropyLoss(weight=weight)
frozen=True

In [ ]:
for child in phobert.children():
  for param in child.parameters():
    param.requires_grad = False

In [ ]:
def train_model(model,data_loader,scheduler_model):
  print("-----------------------Training--------------------------")
  model.train()
  loss_train=0
  final_preds=[]
  final_true=[]
  for idx,(x_batch,y_batch) in enumerate(data_loader):    
    x_batch=x_batch.to(device)
    y_batch=y_batch.to(device)
    outputs=model(x_batch)
    loss=criterion(outputs,y_batch)
    outputs=torch.softmax(outputs,dim=1)
    preds=torch.argmax(outputs,dim=1)
    final_preds.extend(preds.cpu().numpy().tolist())
    final_true.extend(y_batch.cpu().numpy().tolist())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler_model.step()
    loss_train+=loss.item()
    if idx%100==0:
      print(idx,end=' ')
  print()  
  return accuracy_score(final_true,final_preds),loss_train/len(data_loader)


def evaluate(model,data_loader):
  print("------------------------Evaluate---------------------------")
  model.eval()
  loss_val=0
  final_preds=[]
  final_true=[]
  for idx,(x_batch,y_batch) in enumerate(data_loader):
    x_batch=x_batch.to(device)
    y_batch=y_batch.to(device)
    outputs=model(x_batch)
    loss=criterion(outputs,y_batch)
    outputs=torch.softmax(outputs,dim=1)
    preds=torch.argmax(outputs,dim=1)
    loss_val+=loss.item()
    final_preds.extend(preds.cpu().numpy().tolist())
    final_true.extend(y_batch.cpu().numpy().tolist())
    if idx%100==0:
      print(idx,end=' ')

  print()
  return accuracy_score(final_true,final_preds),loss_val/len(data_loader)

In [ ]:
def testing(data_loader):
  model.eval()
  final_preds=[]
  final_true=[]
  with torch.no_grad():
    for idx,(x_batch,y_batch) in enumerate(data_loader):
      x_batch=x_batch.to(device)
      y_batch=y_batch.to(device)
      outputs=model(x_batch)
      outputs=torch.softmax(outputs,dim=1)
      preds=torch.argmax(outputs,dim=1)
      final_preds.extend(preds.cpu().numpy().tolist())
      final_true.extend(y_batch.cpu().numpy().tolist())
      if idx%100==0:
        print(idx,end=' ')
  print()
  print("Accuracy for test:",accuracy_score(final_true,final_preds))

def save_checkpoint(epoch,scheduler_model):
  path='/content/drive/MyDrive/Viettel3/Bert'
  if os.path.exists(path) is False:
    os.mkdir(path)
  
  with open('/content/drive/MyDrive/Viettel3/Bert/history{}.pickle'.format(epoch),'wb') as file:
    pickle.dump(history,file)
  
  print("Save history done")
  model_state={
      'model':model.state_dict(),
      'optimizer':optimizer.state_dict(),
      'scheduler':scheduler_model.state_dict()
  }
  torch.save(model_state,path+'/model{}.pth'.format(epoch))
  print("Save model done")

In [ ]:
from collections import defaultdict
import time 
history=defaultdict(list)
for epoch in range(n_epochs):

  if epoch > 0 and frozen:
    for child in model.bert.children():
      for param in child.parameters():
        param.requires_grad = False
    del scheduler_frozen
    torch.cuda.empty_cache()
    frozen=False 

  start_time=time.time()
  if frozen:
    train_acc,train_loss=train_model(model,train_loader,scheduler_frozen)
  else:
    train_acc,train_loss=train_model(model,train_loader,scheduler)
    
  val_acc,val_loss=evaluate(model,val_loader)
  print(f"Epoch {epoch}----Train acc:{train_acc}----Train loss:{train_loss}---Val acc:{val_acc}---Val loss:{val_loss}---Time:{time.time()-start_time}")
  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)
  if epoch!=0 and epoch%4==0:
    print("--------------Time for testing-----------------")
    testing(test_loader) 
    save_checkpoint(epoch,scheduler)

-----------------------Training--------------------------
0 100 200 300 400 500 600 700 800 
------------------------Evaluate---------------------------
0 100 200 
Epoch 0----Train acc:0.5263586554124093----Train loss:1.3871019316142832---Val acc:0.8070253445975989---Val loss:0.6316838213617768---Time:762.8374083042145
-----------------------Training--------------------------
0 100 200 300 400 500 600 700 800 
------------------------Evaluate---------------------------
0 100 200 
Epoch 1----Train acc:0.8346660743373315----Train loss:0.5294379874358516---Val acc:0.8584556099006966---Val loss:0.45363199442483804---Time:772.7316536903381
-----------------------Training--------------------------
0 100 200 300 400 500 600 700 800 
------------------------Evaluate---------------------------
0 100 200 
Epoch 2----Train acc:0.8630238412557382----Train loss:0.42626490940356393---Val acc:0.8747591522157996---Val loss:0.3856176650637134---Time:770.5834412574768
-----------------------Training----